In [2]:
# Create a new Jupyter notebook cell with the corrected code
%%writefile Meeting_Notes_Generator.ipynb
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {
    "id": "authenticate-google"
   },
   "source": [
    "# Google Docs Meeting Notes Generator\n",
    "Automatically creates formatted meeting notes from markdown using Google Docs API."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "colab": {
     "base_uri": "https://localhost:8080/"
    },
    "id": "authentication-cell",
    "outputId": "b1d6e9d3-0a1b-4d8b-eb6d-7a3c8a80e9f9"
   },
   "outputs": [],
   "source": [
    "# Authentication Cell\n",
    "from google.colab import auth\n",
    "from google.auth import default\n",
    "from googleapiclient.discovery import build\n",
    "from googleapiclient.errors import HttpError\n",
    "import re\n",
    "import datetime\n",
    "\n",
    "# Authenticate and create service\n",
    "auth.authenticate_user()\n",
    "creds, _ = default()\n",
    "service = build('docs', 'v1', credentials=creds)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "doc-creation"
   },
   "outputs": [],
   "source": [
    "# Document Creation Cell\n",
    "doc_title = f\"Product Team Sync - {datetime.datetime.now().strftime('%Y-%m-%d')}\"\n",
    "doc_body = {'title': doc_title}\n",
    "\n",
    "try:\n",
    "    doc = service.documents().create(body=doc_body).execute()\n",
    "    doc_id = doc['documentId']\n",
    "    print(f\"Created new Google Doc: https://docs.google.com/document/d/{doc_id}\")\n",
    "except HttpError as e:\n",
    "    print(f\"Error creating document: {e}\")\n",
    "    raise"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "markdown-processing"
   },
   "outputs": [],
   "source": [
    "# Markdown Conversion Function\n",
    "def convert_markdown_to_requests(markdown):\n",
    "    requests = []\n",
    "    current_end = 1\n",
    "    lines = markdown.strip().split('\\n')\n",
    "\n",
    "    for line in lines:\n",
    "        line = line.rstrip('\\r')\n",
    "        if not line:\n",
    "            continue\n",
    "\n",
    "        # Headings handling\n",
    "        if line.startswith('# '):\n",
    "            text = line[2:].strip()\n",
    "            insert_text = text + '\\n'\n",
    "            requests.extend([\n",
    "                {'insertText': {'location': {'index': current_end}, 'text': insert_text}},\n",
    "                {'updateParagraphStyle': {\n",
    "                    'range': {'startIndex': current_end, 'endIndex': current_end + len(insert_text)},\n",
    "                    'paragraphStyle': {'namedStyleType': 'HEADING_1'},\n",
    "                    'fields': 'namedStyleType'\n",
    "                }}\n",
    "            ])\n",
    "            current_end += len(insert_text)\n",
    "\n",
    "        # Checkbox items\n",
    "        elif line.startswith('- [ ] '):\n",
    "            text = line[6:].strip()\n",
    "            insert_text = text + '\\n'\n",
    "            requests.extend([\n",
    "                {'insertText': {'location': {'index': current_end}, 'text': insert_text}},\n",
    "                {'createParagraphBullets': {\n",
    "                    'range': {'startIndex': current_end, 'endIndex': current_end + len(insert_text)},\n",
    "                    'bulletPreset': 'BULLET_CHECKBOX'\n",
    "                }}\n",
    "            ])\n",
    "            # Bold mentions\n",
    "            for match in re.finditer(r'@\\w+', insert_text):\n",
    "                start, end = match.span()\n",
    "                requests.append({\n",
    "                    'updateTextStyle': {\n",
    "                        'range': {'startIndex': current_end + start, 'endIndex': current_end + end},\n",
    "                        'textStyle': {'bold': True},\n",
    "                        'fields': 'bold'\n",
    "                    }\n",
    "                })\n",
    "            current_end += len(insert_text)\n",
    "\n",
    "        # Regular bullets\n",
    "        elif line.startswith(('- ', '* ')):\n",
    "            text = line[2:].strip()\n",
    "            insert_text = text + '\\n'\n",
    "            requests.extend([\n",
    "                {'insertText': {'location': {'index': current_end}, 'text': insert_text}},\n",
    "                {'createParagraphBullets': {\n",
    "                    'range': {'startIndex': current_end, 'endIndex': current_end + len(insert_text)},\n",
    "                    'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'\n",
    "                }}\n",
    "            ])\n",
    "            current_end += len(insert_text)\n",
    "\n",
    "        # Other content\n",
    "        else:\n",
    "            insert_text = line + '\\n'\n",
    "            requests.append({'insertText': {'location': {'index': current_end}, 'text': insert_text}})\n",
    "            current_end += len(insert_text)\n",
    "\n",
    "    return requests"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "execution-cell"
   },
   "outputs": [],
   "source": [
    "# Execution Cell\n",
    "markdown_content = \"\"\"\n",
    "# Product Team Sync - {date}\n",
    "## Attendees\n",
    "- @Sarah (Product Lead)\n",
    "- @Mike (Engineering)\n",
    "\n",
    "## Action Items\n",
    "- [ ] Finalize roadmap\n",
    "- [ ] Schedule API review\n",
    "\"\"\".format(date=datetime.datetime.now().strftime('%Y-%m-%d'))\n",
    "\n",
    "requests = convert_markdown_to_requests(markdown_content)\n",
    "\n",
    "try:\n",
    "    service.documents().batchUpdate(\n",
    "        documentId=doc_id,\n",
    "        body={'requests': requests}\n",
    "    ).execute()\n",
    "    print(\"Document formatted successfully!\")\n",
    "except HttpError as e:\n",
    "    print(f\"Formatting error: {e}\")"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}

Overwriting Meeting_Notes_Generator.ipynb
